In [ ]:
!pip install keras
!pip install tensorflow
!pip install scipy
!pip install matplotlib
!pip install scikit-image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import keras
from keras.models import Sequential
from keras.datasets import cifar10
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import skimage.util

In [ ]:
from keras.activations import relu, softmax
from keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

def generate_model():
  model = Sequential()
  model.add(InputLayer((32,32,3)))

  model.add(Conv2D(16, 3, activation=relu, padding='same'))
  model.add(MaxPool2D(2))

  model.add(Conv2D(16, 3, activation=relu, padding='same'))
  model.add(Conv2D(16, 3, activation=relu, padding='same'))
  model.add(MaxPool2D(2))

  model.add(Conv2D(32, 3, activation=relu, padding='same'))
  model.add(Conv2D(32, 3, activation=relu, padding='same'))
  model.add(MaxPool2D(2))

  model.add(Flatten())
  model.add(Dense(10, activation=softmax))

  model.compile(Adam(), categorical_crossentropy, metrics=['accuracy'])

  return model

## Questão 2

Carregue as duas bases de dados (CIFAR-10 e SVHN):
1. Ao carregar a base de dados CIFAR-10 você irá notar que y_train e y_test têm duas dimensões. Transforme eles em vetores.
2. Ao carregar a base de dados SVHN, e entender sua estrutura, você irá notar que X_train e X_test estão com as dimensões em uma ordem diferente da que usamos na aula (e que o Keras usa por padrão). Transforme tais arrays para ter a ordem que normalmente usamos (id da imagem, linhas, colunas e canais);
3. Transforme o valor dos pixels das imagens para o intervalo [0; 1];
4. Para cada uma das duas bases, compute:
    - Quantidade de imagens de treino e teste
    - Tamanho (número de linhas, colunas e canais) das imagens
    - Distribuição das classes

Dicas (que podem facilitar muito esse e os próximos itens):
- A função loadmat da biblioteca scipy permite fazer a leitura dos arquivos.mat (base SVHN);
- Converter as imagens (X_train e X_test) para float16 pode ajudar a reduzir o consumo de RAM. O numpy permite fazer tal conversão com um código dotipo: np.float16(X);
- Converter as classes (y_train e y_test) para uint8 permite reduzir o consumo de RAM.

### a) Processando cifar10

In [ ]:
(cifar10_x_train, cifar10_y_train), (cifar10_x_test, cifar10_y_test) = cifar10.load_data()

In [ ]:
cifar10_x_train.shape, cifar10_y_train.shape, cifar10_x_test.shape, cifar10_y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [ ]:
cifar10_y_train = cifar10_y_train.squeeze()
cifar10_y_test = cifar10_y_test.squeeze()

In [ ]:
cifar10_x_train.shape, cifar10_y_train.shape, cifar10_x_test.shape, cifar10_y_test.shape

((50000, 32, 32, 3), (50000,), (10000, 32, 32, 3), (10000,))

### b) Processando svhn

In [ ]:
train = scipy.io.loadmat('/content/drive/My Drive/ICMC/Redes Neurais/train_32x32.mat')
test = scipy.io.loadmat('/content/drive/My Drive/ICMC/Redes Neurais/test_32x32.mat')

In [ ]:
svhn_x_train = train['X']
svhn_y_train = train['y']

svhn_x_test = test['X']
svhn_y_test = test['y']

In [ ]:
svhn_x_train.shape, svhn_y_train.shape, svhn_x_test.shape, svhn_y_test.shape

((32, 32, 3, 73257), (73257, 1), (32, 32, 3, 26032), (26032, 1))

In [ ]:
svhn_x_train = svhn_x_train.transpose(3, 0, 1, 2)
svhn_x_test = svhn_x_test.transpose(3, 0, 1, 2)

svhn_y_train = svhn_y_train.squeeze()
svhn_y_test = svhn_y_test.squeeze()

svhn_y_train -= 1
svhn_y_test -= 1

In [ ]:
svhn_x_train.shape, svhn_y_train.shape, svhn_x_test.shape, svhn_y_test.shape

((73257, 32, 32, 3), (73257,), (26032, 32, 32, 3), (26032,))

In [ ]:
distribution_cifar = np.dstack(np.unique(cifar10_y_train.astype(np.int16), return_counts = True))[0]
distribution_svhn = np.dstack(np.unique(svhn_y_train.astype(np.int16), return_counts = True))[0]

In [ ]:
b = np.zeros((cifar10_y_train.size, cifar10_y_train.max()+1))
b[np.arange(cifar10_y_train.size), cifar10_y_train] = 1
cifar10_y_train = b

b = np.zeros((cifar10_y_test.size, cifar10_y_test.max()+1))
b[np.arange(cifar10_y_test.size), cifar10_y_test] = 1
cifar10_y_test = b

b = np.zeros((svhn_y_train.size, svhn_y_train.max()+1))
b[np.arange(svhn_y_train.size), svhn_y_train] = 1
svhn_y_train = b

b = np.zeros((svhn_y_test.size, svhn_y_test.max()+1))
b[np.arange(svhn_y_test.size), svhn_y_test] = 1
svhn_y_test = b

In [ ]:
cifar10_y_train.shape, cifar10_y_test.shape, svhn_y_train.shape, svhn_y_test.shape

((50000, 10), (10000, 10), (73257, 10), (26032, 10))

### Conversões

In [ ]:
cifar10_x_train = cifar10_x_train.astype(np.float16)
cifar10_x_test = cifar10_x_test.astype(np.float16)
cifar10_y_train = cifar10_y_train.astype(np.uint8)
cifar10_y_test = cifar10_y_test.astype(np.uint8)

svhn_x_train = svhn_x_train.astype(np.float16)
svhn_x_test = svhn_x_test.astype(np.float16)
svhn_y_train = svhn_y_train.astype(np.uint8)
svhn_y_test = svhn_y_test.astype(np.uint8)

### c) Transformando para o intervalo [0,1]

In [ ]:
def normalize(array):
    maxi, mini = array.max(), array.min()
    diff = maxi - mini
    
    if diff == 0:
        diff = 1
    
    return (array - mini) / diff

In [ ]:
cifar10_x_train = normalize(cifar10_x_train)
cifar10_x_test = normalize(cifar10_x_test)

svhn_x_train = normalize(svhn_x_train)
svhn_x_test = normalize(svhn_x_test)

### d) 

#### cifar10

In [ ]:
print('Quantidade de imagens de treino:', cifar10_x_train.shape[0])
print('Quantidade de imagens de teste:', cifar10_x_test.shape[0])
print('Tamanho (número de linhas, colunas e canais) das imagens', cifar10_x_train[0].shape)
print('Distribuição das classes\n', distribution_cifar)

Quantidade de imagens de treino: 50000
Quantidade de imagens de teste: 10000
Tamanho (número de linhas, colunas e canais) das imagens (32, 32, 3)
Distribuição das classes
 [[   0 5000]
 [   1 5000]
 [   2 5000]
 [   3 5000]
 [   4 5000]
 [   5 5000]
 [   6 5000]
 [   7 5000]
 [   8 5000]
 [   9 5000]]


#### svhn

In [ ]:
print('Quantidade de imagens de treino:', svhn_x_train.shape[0])
print('Quantidade de imagens de teste:', svhn_x_test.shape[0])
print('Tamanho (número de linhas, colunas e canais) das imagens', svhn_x_train[0].shape)
print('Distribuição das classes\n', distribution_svhn)

Quantidade de imagens de treino: 73257
Quantidade de imagens de teste: 26032
Tamanho (número de linhas, colunas e canais) das imagens (32, 32, 3)
Distribuição das classes
 [[    0 13861]
 [    1 10585]
 [    2  8497]
 [    3  7458]
 [    4  6882]
 [    5  5727]
 [    6  5595]
 [    7  5045]
 [    8  4659]
 [    9  4948]]


É notável que o cifar10 está muito mais balanceado que o svhn

## Questão 3

Vamos gerar versões ruidosas das bases de imagens e ver como isso afeta sua qualidade visual.
1. Gerar versões das duas bases de dados afetadas por ruído gaussiano com variâncias de 0.001 e 0.01, respectivamente;
2. Mostrar uma imagem de cada uma das classes em 3 versões (original, Gauss 0.001 e Gauss 0.01). Isso deve ser feito para as duas bases (CIFAR-10 e SVHN).

O ruído gaussiano gera imagens parecidas com as do seguinte exemplo, no qual temos: as imagens originais na primeira linha, imagens com ruído gaussiano (var 0.001) na segunda linha e imagens com ruído gaussiano (var 0.01) na terceira linha. 

Dica:
- Para gerar as imagens com ruído gaussiano use a função random_noise da biblioteca scikit-image. Tal função de ser usada da seguinte forma: random_noise(img_original, mode='gaussian', var=0.01);

In [ ]:
class NoisyImage:
    def __init__(self, original):
        self.original = original
        self.noisy0001 = self.apply_noise(0.001)
        self.noisy001 = self.apply_noise(0.01)
    
    def apply_noise(self, var):
        # return skimage.util.random_noise(self.original, mode='gaussian', var=var)
        self.noisy = self.original.copy()
        for i in np.arange(self.original.shape[0]):
            self.noisy[i] = skimage.util.random_noise(self.original[i], mode='gaussian', var=var)
        return self.noisy

In [ ]:
%%time

cifar10_x_train = NoisyImage(cifar10_x_train)
cifar10_x_test = NoisyImage(cifar10_x_test)

svhn_x_train = NoisyImage(svhn_x_train)
svhn_x_test = NoisyImage(svhn_x_test)

CPU times: user 1min 30s, sys: 318 ms, total: 1min 31s
Wall time: 1min 30s


## Questão 4

Treinar uma versão do modelo para cada versão das bases de dados. Ou seja, você deve treinar os seguintes seis modelos:
1. Modelo treinado na CIFAR-10 original;
2. Modelo treinado na CIFAR-10 Gauss 0.001;
3. Modelo treinado na CIFAR-10 Gauss 0.01;


1. Modelo treinado na SVHN original;
2. Modelo treinado na SVHN Gauss 0.001;
3. Modelo treinado na SVHN Gauss 0.01;

Todos os modelos devem ser treinado usando:
- Adam como otimizador;
- Por 10 epocas;
- Os demais parâmetros devem ser deixados como padrão.

Dica:
- Você pode salvar os modelos e depois carregá-los novamente, com as
funções save e load_model. Verifique a documentação do Keras.

In [ ]:
model = generate_model()
model.fit(cifar10_x_train.original, cifar10_y_train, epochs=10)
model.save('/content/drive/My Drive/ICMC/Redes Neurais/cifar10_original')

model.fit(cifar10_x_train.noisy0001, cifar10_y_train, epochs=10)
model.save('/content/drive/My Drive/ICMC/Redes Neurais/cifar10_noisy0001')

model.fit(cifar10_x_train.noisy001, cifar10_y_train, epochs=10)
model.save('/content/drive/My Drive/ICMC/Redes Neurais/cifar10_noisy001')



model.fit(svhn_x_train.original, svhn_y_train, epochs=10)
model.save('/content/drive/My Drive/ICMC/Redes Neurais/svhn_original')

model.fit(svhn_x_train.noisy0001, svhn_y_train, epochs=10)
model.save('/content/drive/My Drive/ICMC/Redes Neurais/svhn_noisy0001')

model.fit(svhn_x_train.noisy001, svhn_y_train, epochs=10)
model.save('/content/drive/My Drive/ICMC/Redes Neurais/svhn_noisy001')

Epoch 1/10
1563/1563 [==============================] - 20s 4ms/step - loss: 1.7922 - accuracy: 0.3494
Epoch 2/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.2726 - accuracy: 0.5431
Epoch 3/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1045 - accuracy: 0.6111
Epoch 4/10
1563/1563 [==============================] - 6s 4ms/step - loss: 1.0002 - accuracy: 0.6489
Epoch 5/10
1563/1563 [==============================] - 6s 4ms/step - loss: 0.9180 - accuracy: 0.6799
Epoch 6/10
1563/1563 [==============================] - 7s 4ms/step - loss: 0.8685 - accuracy: 0.6960
Epoch 7/10
1563/1563 [==============================] - 6s 4ms/step - loss: 0.8408 - accuracy: 0.7074
Epoch 8/10
1563/1563 [==============================] - 7s 4ms/step - loss: 0.7875 - accuracy: 0.7243
Epoch 9/10
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7558 - accuracy: 0.7372
Epoch 10/10
1563/1563 [==============================] - 6s 4ms/step - loss: 0.72

## Questão 5

Agora, vamos tentar entender os impactos do ruído na acurácia dos modelos fazendo os seguintes experimentos:
1. Calcular a acurácia dos modelos em todas as versões da base de teste. Por exemplo, para o modelo treinado com os dados de treinamento da CIFAR-10 original devemos computar sua acurácia nas bases de teste original, Gauss 0.001 e Gauss 0.01. Ou seja, para cada um dos modelos vamos computar 3 acurácias;
2. Com bases nesses resultados, discuta qual das duas afirmações mais explica as variações de acurácia dos nossos experimentos:
  1. O ruído torna o problema mais difícil, logo fica mais difícil para o modelo aprender com dados ruidosos;
  2. Quando um modelo é exposto a dados bastante diferentes dos de seu treinamento (e.g. com bastante ruído) sua acurácia pode cair.

In [ ]:
models_cifar = [
    'cifar10_original',
    'cifar10_noisy0001',
    'cifar10_noisy001',
]

models_svhn = [
    'svhn_original',
    'svhn_noisy0001',
    'svhn_noisy001',
]

for model_folder in models_cifar:
    model = keras.models.load_model('/content/drive/My Drive/ICMC/Redes Neurais/' + model_folder)

    cifar10_original = model.evaluate(cifar10_x_test.original, cifar10_y_test, use_multiprocessing=True, return_dict=True, verbose=False)
    cifar10_noisy0001 = model.evaluate(cifar10_x_test.noisy0001, cifar10_y_test, use_multiprocessing=True, return_dict=True, verbose=False)
    cifar10_noisy001 = model.evaluate(cifar10_x_test.noisy001, cifar10_y_test, use_multiprocessing=True, return_dict=True, verbose=False)

    print('Results for model', model_folder)
    print('cifar10_original:', cifar10_original)
    print('cifar10_noisy0001:', cifar10_noisy0001)
    print('cifar10_noisy001:', cifar10_noisy001)
    print('Média:', (cifar10_original['accuracy'] + cifar10_noisy0001['accuracy'] + cifar10_noisy001['accuracy']) / 3)

print('-' * 80)

for model_folder in models_svhn:
    model = keras.models.load_model('/content/drive/My Drive/ICMC/Redes Neurais/' + model_folder)

    svhn_original = model.evaluate(svhn_x_test.original, svhn_y_test, use_multiprocessing=True, return_dict=True, verbose=False)
    svhn_noisy0001 = model.evaluate(svhn_x_test.noisy0001, svhn_y_test, use_multiprocessing=True, return_dict=True, verbose=False)
    svhn_noisy001 = model.evaluate(svhn_x_test.noisy001, svhn_y_test, use_multiprocessing=True, return_dict=True, verbose=False)

    print('Results for model', model_folder)
    print('svhn_original:', svhn_original)
    print('svhn_noisy0001:', svhn_noisy0001)
    print('svhn_noisy001:', svhn_noisy001)
    print('Média:', (svhn_original['accuracy'] + svhn_noisy0001['accuracy'] + svhn_noisy001['accuracy']) / 3)

Results for model cifar10_original
cifar10_original: {'loss': 0.9485189318656921, 'accuracy': 0.6812000274658203}
cifar10_noisy0001: {'loss': 1.058789610862732, 'accuracy': 0.6439999938011169}
cifar10_noisy001: {'loss': 1.919073462486267, 'accuracy': 0.4300000071525574}
Média: 0.5850666761398315
Results for model cifar10_noisy0001
cifar10_original: {'loss': 0.9699365496635437, 'accuracy': 0.6948000192642212}
cifar10_noisy0001: {'loss': 0.9673864245414734, 'accuracy': 0.6930000185966492}
cifar10_noisy001: {'loss': 1.3808842897415161, 'accuracy': 0.5800999999046326}
Média: 0.6559666792551676
Results for model cifar10_noisy001
cifar10_original: {'loss': 1.1915020942687988, 'accuracy': 0.6237000226974487}
cifar10_noisy0001: {'loss': 1.1636205911636353, 'accuracy': 0.6284999847412109}
cifar10_noisy001: {'loss': 1.1198492050170898, 'accuracy': 0.6491000056266785}
Média: 0.6337666710217794
--------------------------------------------------------------------------------
Results for model svhn_

### Hipótese 1

Para validar a hipótese 1

- O ruído torna o problema mais difícil, logo fica mais difícil para o modelo aprender com dados ruidosos;

Precisamos comparar o quanto a acurácia geral dos modelos treinados com imagens ruidosas caiu.

Como pode ser observado acima, a acurácia geral não só não caiu muito como aumentou expressivamente para os testes nas bases de teste com ruído. Se analisarmos a média, podemos ver que ela aumentou. 

Em suma, se o problema que você está tentando resolver estiver propenso a ruidos, não adianta treinar numa base pura sem ruídos. É necessário que a base utilizada para treinamento esteja condizente com o contexto em que o classificador será utilizado

### Hipótese 2

Para validar a hipótese 2

- Quando um modelo é exposto a dados bastante diferentes dos de seu treinamento (e.g. com bastante ruído) sua acurácia pode cair.

É necessário considerar o modelo treinado com as imagens originais e testado com as imagens com ruído:

Results for model cifar10_original
- cifar10_original: {'loss': 0.9485189318656921, 'accuracy': 0.6812000274658203}
- cifar10_noisy0001: {'loss': 1.058789610862732, 'accuracy': 0.6439999938011169}
- cifar10_noisy001: {'loss': 1.919073462486267, 'accuracy': 0.4300000071525574}

Results for model svhn_original
- svhn_original: {'loss': 0.396482914686203, 'accuracy': 0.8864858746528625}
- svhn_noisy0001: {'loss': 0.4289434254169464, 'accuracy': 0.8763445019721985}
- svhn_noisy001: {'loss': 0.9148530960083008, 'accuracy': 0.7098954916000366}

Como visto acima, realmente a acurácia cai quando o modelo treinado com imagens sem ruído é exposto a imagens com ruído. Aparentemente quanto mais ruído mais ela cai. Isso reforça a conclusão que cheguei na análise da hipótese 1. Se seu contexto será de imagens ruidósas, não adianta treinar com imagens limpas.



### Conclusão

A hipóstese 2 está mais condizente, logo, no geral, parece ser a que melhor explica a situação